# 1- PLATEFORME DE PRÉDICTION DU COURS DES ACTIONS

Ce document sert à effectuer des prédictions sur le cours des actions côtés sur la BRVM. La première étape est d'initialiser les données sur la plateforme afin de pouvoir effectuer les calculs.

* Pour exécuter les lignes de code, il suffit de taper sur Shift puis Entrée, vous pouvez aussi utiliser la fonction Run All Cells.
* Après avoir exécuter le code commençant par !pip install, supprimer la ligne de code.

## 1.1 - INITIALISATION DES DONNÉES

Cette étape consiste à importer les modules qui vont servir à une exécution correcte du script.

In [13]:
# Intallation module

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
sns.set_style("whitegrid")
import math

# Modules for the buttons and widgets 
import ipywidgets as widgets
from ipywidgets import Button, Layout, Text,  HBox
from ipywidgets import Dropdown
from ipywidgets import Output
from IPython.display import display

Code à exécuter une fois puis supprimer le code

In [ ]:
# Code à exécuter pour installation puis supprimer le code
! pip install keras
! pip install --upgrade tensorflow

### 1.1-A) Importer des données historiques à partir d'Excel

La base de donnée doit suivre un certain format (Voir Manuel de Procédure).

Pour importer votre base de données il faudra appuyer sur SHIFT + ENTRÉE qui permet d'exécuter le code et de choisir votre fichier directement de votre odinateur. 

Sinon, une alternative est de copier le code de dépannage et de l'exécuter.

Pour cela, il faudra mettre les informations sur le paths (chemin) à suivre pour avoir accès au fichier.
* Si vous avez un windows, utilisez file_path = "C:\\Users\\votrenom\\emplacementdufichier\\nomdufichier.xlsx"
* Si vous avez un macOs ou Linux, exemple de path : file_path= "/Users/arambengue/Documents/CGF 2024/Analyse python 46 titres/Datamonthly.xlsx"

* Vous pouvez utiliser le code suivant en cas de bug pour vous dépanner, pensez à modifier le path en fonction de l'emplacement de votre document: 

file_path= "/Users/arambengue/Documents/pricesWORGT.xlsx"

data = pd.read_excel(file_path,header=0,index_col=0)

data.index = pd.to_datetime(data.index, dayfirst=True)

data

Reminder : Si vous utilisez le code ci-dessus, pensez à modifier le file path.

In [ ]:
import pandas as pd
from PyQt5.QtWidgets import QApplication, QFileDialog

def upload_excel():
    """Prompts the user to select an Excel file and returns the file path."""
    app = QApplication([])
    file_path, _ = QFileDialog.getOpenFileName(None, "Select Excel File", "", "Excel files (*.xlsx *.xls)")
    app.exit()
    return file_path

def read_excel(file_path):
  """Reads the selected Excel file into a pandas DataFrame."""
  if file_path:
    data = pd.read_excel(file_path,header=0,index_col=0)
    return data
  else:
    print("No file selected")
    
# Usage
file_path = upload_excel()
if file_path:
    data = read_excel(file_path)
    data.index = pd.to_datetime(data.index, dayfirst=True)
    print(data.head())


## 1.2 - PRÉDICTIONS DES COURS SUR 30 JOURS

Le modèle utilisé pour les prédictions sur 30 jours est le modèle LSTM. Il permet de prévoir les prix futurs des actions en analysant les données historiques des cours. Les modèles LSTM peuvent apprendre des comportements, caractéristiques et tendances pour prédire les mouvements futurs des cours. Les modèles LSTM peuvent aussi saisir les relations entre des événements séparés par de longs intervalles de temps, ce qui est important pour comprendre les tendances des cours boursiers.


- Vous pourrez voir l'efficacité du modèle en cliquant sur le bouton vert "Prédictions sur le testing data". Le MSE (Mean Squared Error) et le RMSE (Root Mean Squared Error) seront présentés. Ce sont des mesures courantes utilisées pour évaluer les performances des modèles de régression, y compris les modèles LSTM. Elles mesurent la différence quadratique moyenne entre les valeurs prédites et les valeurs réelles.
- Vous pourrez voir les prédictions du modèle sur 30 jours à partir de la date de fin de votre base de donnée. Il est donc conseillé d'utiliser une base de données journalière, à jour et exhaustive (voir 1.1-A Importer données Excel). 
- Vous pouvez sélectionner le titre dont vous souhaitez avoir les prédictions.



In [ ]:
stock_options = data.columns.tolist()
stock_dropdown = widgets.Dropdown(options=stock_options, description='Actions:')


    
def on_test():
    dataset = data.loc[:,[stock_dropdown.value]]
    training_data_len = math.ceil(len(dataset)*.8)
    train, test = dataset[:training_data_len], dataset[training_data_len:]
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset)
    # Create the training data set
    train_data = scaled_data[0:training_data_len,:]
    ## Split the data into train and test
    x_train = []
    y_train = []
    
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
        
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    
    # Model trained on test 
    model = Sequential()
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=5)
    
    test_data = scaled_data[training_data_len-60: , :]
    #Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
    
    x_test = np.array(x_test)
    lstm_predictions = model.predict(x_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    # Error Measures
    mse = ((lstm_predictions-y_test).mean(axis=0))**2
    rmse = np.sqrt(mse)
    print("\n")
    print(f"Le Mean Squared Error est:",mse.values[0])
    print(f"Le Root Mean Squared Error est:",rmse.values[0])

    
    # Plot the data
    train = data[:training_data_len]
    valid = data[training_data_len:].copy()
    valid['Predictions'] = lstm_predictions
    plt.figure(figsize=(16,8))
    plt.title(f"TESTING: Cours réel de {stock_dropdown.value} vs. prédictions du modèle LSTM")
    plt.xlabel('Date', fontsize=18)
    plt.ylabel(f"Cours de clôture de {stock_dropdown.value}", fontsize=18)
    plt.plot(train[stock_dropdown.value])
    plt.plot(valid[[stock_dropdown.value,'Predictions']])
    plt.legend(['training data', 'testing data', 'Predictions'], loc = 'lower right')
    plt.show()
    
    # Comparaison
    above = True
    below = True

    for y1_val, y2_val in zip(valid[stock_dropdown.value],valid['Predictions']):
        if y1_val < y2_val:
            above = False
        if y1_val > y2_val:
            below = False
    if above:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} sont constamment en dessous des données Test. Les prédictions du modèle ont donc une marge d'erreur.")
    elif below:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} sont constamment au dessus des données Test. Les prédictions du modèle ont donc une marge d'erreur.")
    else:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} et les données Test 'réelles' se superposent. Le modèle a un bon fit.")
    
    
def testing():
    on_test()

testing_button = Button(description="Prédictions sur le Testing Set")
testing_button.style.button_color = 'lightgreen'
testing_button.layout.width = '240px'
testing_button.layout.height = '40px'

output1 = widgets.Output()

def on_button_clicked1(b):
 with output1:
    output1.clear_output()
    testing()
    
testing_button.on_click(on_button_clicked1)    


def forecasting():
    dataset = data.loc[:,[stock_dropdown.value]]
    training_data_len = math.ceil(len(dataset)*.8)
    train, test = dataset[:training_data_len], dataset[training_data_len:]
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dataset)
    # Create the training data set
    train_data = scaled_data[0:training_data_len,:]
    ## Split the data into train and test
    x_train = []
    y_train = []
    
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
        
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    
    model = Sequential()
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(30))
  # Change to 30 units for 30-day forecast

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=5)
    
    # Forecast for 30 days
    sequence_length = 60 
    forecast_days = 30

    last_60_days = scaled_data[-sequence_length:]  # Last 60 days of scaled data
    forecast_input = np.array([last_60_days])
    forecast_scaled = model.predict(forecast_input)
    forecast = scaler.inverse_transform(forecast_scaled)
    print("\n")
    # Print the forecasted closing prices for 30 days
    print(f"Prix de clôture de {stock_dropdown.value} sur les 30 prochains jours:")
    for i in range(forecast_days):
        print(f"Journée de cotation {i+1}: {forecast[0, i]:.2f}")
        
    # Visualize actual vs. predicted
    
    plt.figure(figsize=(16, 8))
    plt.plot(data.index[-60:], data[stock_dropdown.value].values[-60:], label='Actual')
    plt.plot(pd.date_range(start=data.index[-1], periods=forecast_days, freq='D'), forecast.flatten(), label='Forecast')
    plt.xlabel('Date',fontsize=18)
    plt.ylabel(f"Cours Historique de {stock_dropdown.value}",fontsize=18)
    plt.title(f"Cours actuel de {stock_dropdown.value} vs. Cours de clôture prévus")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Comparaison
    above = True
    below = True

    for x1, x2,y3_val, y4_val in zip(data.index[-60:],pd.date_range(start=data.index[-1], periods=forecast_days, freq='D'),data[stock_dropdown.value].values[-60:],forecast.flatten()):
        if y3_val < y4_val:
            above = False
        if y3_val > y4_val:
            below = False
    if above:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} sont constamment en dessous des cours précédents. Le cours de l'action {stock_dropdown.value} est prévu à la baisse sur les 30 prochains jours.")
    elif below:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} sont constamment au dessus des cours précédents. Le cours de l'action {stock_dropdown.value} est prévu à la hausse sur les 30 prochains jours.")
    else:
        print(f"Les prédictions des cours journaliers de {stock_dropdown.value} et les cours précédents sont au même niveau.Le cours de l'action {stock_dropdown.value} est prévu en range sur les 30 prochains jours ")
    
    
def forecast():
    forecasting()

forecast_button = Button(description="Prédictions sur les 30 prochains jours")
forecast_button.style.button_color = 'orange'
forecast_button.layout.width = '260px'
forecast_button.layout.height = '40px'

output2 = widgets.Output()

def on_button_clicked2(b):
 with output2:
    output2.clear_output()
    forecast()
    
forecast_button.on_click(on_button_clicked2)    

hbox = HBox(children=[testing_button,forecast_button])
display(stock_dropdown,hbox,output1,output2)

Outil CGF BOURSE...